In [1]:
# load libraries
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import re

In [2]:
# load data
directory = "C:\\Users\\joshg\\OneDrive\\Desktop\\code_pudding\\code_pudding_v2\\code_pudding2"
df = pd.read_csv(directory + "\speed_dating_data.csv", encoding='latin1')

In [3]:
# viewing a sample of the data
print(df.info())
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB
None


,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
99,10,10.0,0,19,1,1,10,2,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5287,355,15.0,0,29,2,14,18,20,20.0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791,55,16.0,1,32,2,2,19,3,NaN,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7710,522,14.0,0,27,2,21,22,4,4.0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
815,56,1.0,0,1,1,3,10,5,NaN,5,...,9.0,10.0,8.0,9.0,7.0,NaN,NaN,NaN,NaN,NaN
4404,291,19.0,1,38,2,11,21,21,21.0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,26,6.0,0,11,2,2,16,13,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6048,395,17.0,0,33,2,15,19,11,11.0,8,...,9.0,10.0,9.0,9.0,7.0,8.0,9.0,8.0,9.0,7.0
2475,180,7.0,0,13,1,8,10,2,2.0,3,...,9.0,7.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN
7658,520,12.0,0,23,2,21,22,9,9.0,19,...,8.0,9.0,9.0,7.0,5.0,7.0,9.0,7.0,6.0,6.0


In [4]:
# get rid of a bad entry
df = df.dropna(subset='pid')

### Feature Engineering

In [5]:
# number of dates per attendee
dates_count = df.groupby('iid')['iid'].count()

def get_dates_count(obs):
    return dates_count[obs['iid']]

df['dates_count'] = df.apply(get_dates_count, axis=1)

In [6]:
# check to see if above matches 'round'
df['sanity'] = df['dates_count'] - df['round']
df[df['sanity'] != 0][['iid', 'wave', 'pid', 'round', 'dates_count', 'sanity']]

,iid,wave,pid,round,dates_count,sanity
1746,122,5,112.0,10,9,-1
1747,122,5,113.0,10,9,-1
1748,122,5,114.0,10,9,-1
1749,122,5,115.0,10,9,-1
1750,122,5,116.0,10,9,-1
...,...,...,...,...,...,...
6799,453,17,435.0,11,10,-1
6800,453,17,436.0,11,10,-1
6801,453,17,437.0,11,10,-1
6802,453,17,438.0,11,10,-1


In [7]:
matches_given = df.groupby('iid')['match'].sum()

def get_matches_given(obs):
    return matches_given[obs['iid']]

df['matches_given'] = df.apply(get_matches_given, axis=1)

In [8]:
matches = {}

for iid in df['iid'].unique():
    matches[iid] = []

def get_matches(obs):
    if obs['match']:
        matches[obs['iid']].append(obs['pid'])

df.apply(get_matches, axis=1)

matches

{1: [13.0, 14.0, 15.0, 19.0],
 2: [14.0, 19.0],
 3: [],
 4: [14.0, 19.0],
 5: [14.0, 19.0],
 6: [14.0, 19.0],
 7: [14.0, 19.0],
 8: [12.0, 13.0, 14.0, 15.0, 16.0, 18.0, 19.0, 20.0],
 9: [12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 19.0],
 10: [13.0, 19.0],
 11: [],
 12: [8.0, 9.0],
 13: [1.0, 8.0, 9.0, 10.0],
 14: [1.0, 2.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 15: [1.0, 8.0, 9.0],
 16: [8.0, 9.0],
 17: [9.0],
 18: [8.0],
 19: [1.0, 2.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 20: [8.0],
 21: [],
 22: [44.0, 45.0],
 23: [43.0, 44.0, 45.0],
 24: [],
 25: [],
 26: [],
 27: [46.0, 50.0],
 28: [46.0],
 29: [44.0, 45.0],
 30: [49.0, 55.0],
 31: [48.0, 49.0, 50.0, 55.0],
 32: [],
 33: [],
 34: [46.0, 49.0],
 35: [45.0, 46.0, 49.0, 50.0, 53.0],
 36: [49.0, 51.0],
 37: [44.0, 52.0],
 38: [49.0, 50.0, 55.0],
 39: [47.0],
 40: [],
 41: [],
 42: [],
 43: [23.0],
 44: [22.0, 23.0, 29.0, 37.0],
 45: [22.0, 23.0, 29.0, 35.0],
 46: [27.0, 28.0, 34.0, 35.0],
 47: [39.0],
 48: [31.0],
 49: [30.0, 31.0, 34.0, 35.0, 36

In [9]:
def get_dec_o2(obs):
    if obs['iid'] in matches[obs['pid']]:
        return 1
    else:
        return 0

df['dec_o2'] = df.apply(get_dec_o2, axis=1)

In [10]:
matches_received = df.groupby('iid')['dec_o2'].sum()

def get_matches_received(obs):
    return matches_received[obs['iid']]

df['matches_received'] = df.apply(get_matches_received, axis=1)

In [11]:
def get_mutual_match(obs):
    return obs['match'] and obs['dec_o2']

df['mutual_match'] = df.apply(get_mutual_match, axis=1)

In [12]:
mutual_count = df.groupby('iid')['mutual_match'].sum()

def get_mutual_count(obs):
    return mutual_count[obs['iid']]
    
df['mutual_count'] = df.apply(get_mutual_count, axis=1)

In [13]:
df['match_given_rate'] = df['matches_given'] / df['dates_count']
df['match_receive_rate'] = df['matches_received'] / df['dates_count']
df['mutual_rate'] = df['mutual_count'] / df['matches_given']

In [14]:
career_codes = {1.0: 'Lawyer',
                2.0: 'Academic/Research',
                3.0: 'Psychologist',
                4.0: 'Doctor/Medicine',
                5.0: 'Engineer',
                6.0: 'Creative Arts/Entertainment',
                7.0: 'Banking/Consulting/Finance/Marketing/Business/CEO/Entrepreneur/Admin',
                8.0: 'Real Estate',
                9.0: 'International/Humanitarian Affairs',
                10.0: 'Undecided',
                11.0: 'Social Work',
                12.0: 'Speech Pathology',
                13.0: 'Politics',
                14.0: 'Pro sports/Athletics',
                15.0: 'Other',
                16.0: 'Journalism',
                17.0: 'Architecture'}

career_codes = pd.Series(career_codes)

def decode_careers(obs):
    if obs['career_c'] in career_codes.keys():
        return career_codes[obs['career_c']]
    else:
        return 'Other'

df['career_name'] = df.apply(decode_careers, axis=1)

In [15]:
def demand_satisfaction(obs):
    satis = obs['satis_2']
    if satis <= 2:
        return 'Dissatisfied'
    elif 2 < satis <=4:
        return 'Somewhat Dissatisfied'
    elif 4 < satis <= 6:
        return 'Somewhat Satisfied'
    elif 6 < satis <= 8:
        return 'Satisfied'
    elif 8 < satis:
        return 'Very Satisfied'

df['satisfaction'] = df.apply(demand_satisfaction, axis=1)

In [16]:
df_uniq_id = df.drop_duplicates('iid')
df_uniq_id

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,matches_given,dec_o2,matches_received,mutual_match,mutual_count,match_given_rate,match_receive_rate,mutual_rate,career_name,satisfaction
0,1,1.0,0,1,1,1,10,7,NaN,4,...,4,0,4,0,4,0.400000,0.400000,1.0,Other,Somewhat Satisfied
10,2,2.0,0,3,1,1,10,3,NaN,10,...,2,0,2,0,2,0.200000,0.200000,1.0,Other,Somewhat Satisfied
20,3,3.0,0,5,1,1,10,9,NaN,6,...,0,0,0,0,0,0.000000,0.000000,NaN,Other,None
30,4,4.0,0,7,1,1,10,6,NaN,3,...,2,0,2,0,2,0.200000,0.200000,1.0,Lawyer,Somewhat Dissatisfied
40,5,5.0,0,9,1,1,10,4,NaN,1,...,2,0,2,0,2,0.200000,0.200000,1.0,Lawyer,Satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8268,548,18.0,1,36,2,21,22,21,9.0,13,...,5,1,5,1,5,0.227273,0.227273,1.0,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Dissatisfied
8290,549,19.0,1,38,2,21,22,21,7.0,15,...,5,0,5,0,5,0.227273,0.227273,1.0,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Satisfied
8312,550,20.0,1,40,2,21,22,21,20.0,2,...,4,0,4,0,4,0.181818,0.181818,1.0,Banking/Consulting/Finance/Marketing/Business/...,Satisfied
8334,551,21.0,1,42,2,21,22,21,21.0,1,...,2,0,2,0,2,0.090909,0.090909,1.0,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Dissatisfied


In [17]:
career_counts = df_uniq_id[['career_name']].value_counts().reset_index()
career_counts.columns = ['career_name', 'count']

career_counts

,career_name,count
0,Academic/Research,152
1,Banking/Consulting/Finance/Marketing/Business/...,144
2,Lawyer,47
3,Creative Arts/Entertainment,46
4,International/Humanitarian Affairs,35
5,Doctor/Medicine,26
6,Undecided,25
7,Other,17
8,Psychologist,17
9,Engineer,14


In [18]:
career_success = df_uniq_id.groupby('career_name')['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
career_success.columns = ['career_name', 'match_receive_rate']
career_success[['career_name', 'match_receive_rate']]

,career_name,match_receive_rate
0,Pro sports/Athletics,0.300000
1,Psychologist,0.232177
2,Lawyer,0.207267
3,Social Work,0.192633
4,Banking/Consulting/Finance/Marketing/Business/...,0.184912
5,Journalism,0.181818
6,Engineer,0.179563
7,Doctor/Medicine,0.161584
8,Creative Arts/Entertainment,0.161296
9,Other,0.157254


In [19]:
age_count = df_uniq_id.groupby(['gender', 'age'])['match_receive_rate'].count()
age_count

gender  age 
0       19.0     1
        20.0     3
        21.0    14
        22.0    24
        23.0    32
        24.0    29
        25.0    29
        26.0    31
        27.0    28
        28.0    19
        29.0    17
        30.0    14
        31.0     3
        32.0     3
        33.0     7
        34.0     7
        35.0     4
        36.0     2
        38.0     1
        55.0     1
1       18.0     1
        19.0     1
        20.0     2
        21.0     8
        22.0    20
        23.0    24
        24.0    27
        25.0    26
        26.0    24
        27.0    40
        28.0    28
        29.0    23
        30.0    22
        31.0     4
        32.0    10
        33.0     5
        34.0     4
        36.0     2
        37.0     1
        39.0     1
        42.0     1
Name: match_receive_rate, dtype: int64

In [20]:
age_success = df_uniq_id.groupby('age')['match_receive_rate'].mean()
age_success

age
18.0    0.000000
19.0    0.250000
20.0    0.277778
21.0    0.242301
22.0    0.155225
23.0    0.167488
24.0    0.141090
25.0    0.232406
26.0    0.180530
27.0    0.152766
28.0    0.183927
29.0    0.160904
30.0    0.149701
31.0    0.140873
32.0    0.104706
33.0    0.124025
34.0    0.139322
35.0    0.157937
36.0    0.112847
37.0    0.000000
38.0    0.105263
39.0    0.111111
42.0    0.200000
55.0    0.000000
Name: match_receive_rate, dtype: float64

In [21]:
df.groupby('wave')['match'].mean()

wave
1     0.310000
2     0.101974
3     0.130000
4     0.200617
5     0.300000
6     0.200000
7     0.167969
8     0.180000
9     0.155000
10    0.185185
11    0.145125
12    0.107143
13    0.177778
14    0.175000
15    0.184211
16    0.250000
17    0.171429
18    0.083333
19    0.164444
20    0.119048
21    0.148760
Name: match, dtype: float64

In [22]:
df.iloc[:,-11:].sample(20)

,sanity,matches_given,dec_o2,matches_received,mutual_match,mutual_count,match_given_rate,match_receive_rate,mutual_rate,career_name,satisfaction
1242,0,2,0,2,0,2,0.111111,0.111111,1.0,Academic/Research,Dissatisfied
6148,0,6,1,6,1,6,0.333333,0.333333,1.0,Banking/Consulting/Finance/Marketing/Business/...,None
7752,0,14,0,14,0,14,0.636364,0.636364,1.0,Doctor/Medicine,Satisfied
4153,0,5,1,5,1,5,0.238095,0.238095,1.0,Academic/Research,Somewhat Satisfied
8184,0,3,0,3,0,3,0.136364,0.136364,1.0,Doctor/Medicine,Satisfied
4249,0,1,0,1,0,1,0.047619,0.047619,1.0,Creative Arts/Entertainment,Somewhat Satisfied
2566,0,0,0,0,0,0,0.000000,0.000000,NaN,International/Humanitarian Affairs,Somewhat Dissatisfied
6281,0,2,0,2,0,2,0.111111,0.111111,1.0,Creative Arts/Entertainment,Somewhat Satisfied
5334,0,2,0,2,0,2,0.111111,0.111111,1.0,Banking/Consulting/Finance/Marketing/Business/...,None
8313,0,4,0,4,0,4,0.181818,0.181818,1.0,Banking/Consulting/Finance/Marketing/Business/...,Satisfied


In [23]:
scorecard_ratings = ['attr', 'sinc', 'intel', 'fun', 'amb']
others_ratings = ['attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o']
self_rate_before = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
self_rate_day_after = ['attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2']
self_rate_weeks_after = ['attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3']
self_rate_before = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
others_ratings_avg = df.groupby('pid')[['attr', 'sinc', 'intel', 'fun', 'amb']].mean()


In [24]:
self_ratings = df[['iid','attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2']]
self_ratings = self_ratings.drop_duplicates(subset='iid')

all_ratings = self_ratings.merge(others_ratings_avg, left_on='iid', right_on='pid')

all_ratings['diff_attr'] = all_ratings['attr'] - all_ratings['attr3_2']
all_ratings['diff_sinc'] = all_ratings['sinc'] - all_ratings['sinc3_2']
all_ratings['diff_intel'] = all_ratings['intel'] - all_ratings['intel3_2']
all_ratings['diff_fun'] = all_ratings['fun'] - all_ratings['fun3_2']
all_ratings['diff_amb'] = all_ratings['amb'] - all_ratings['amb3_2']

all_ratings.loc[:,'diff_attr':].describe()

,diff_attr,diff_sinc,diff_intel,diff_fun,diff_amb
count,485.000000,485.000000,485.000000,485.000000,485.000000
mean,-0.901542,-0.716509,-0.824920,-1.137294,-0.643552
std,1.532698,1.624215,1.310515,1.518988,1.816832
min,-5.733333,-4.437500,-5.125000,-5.888889,-4.600000
25%,-1.777778,-1.904762,-1.666667,-2.166667,-2.000000
50%,-0.888889,-0.875000,-0.842105,-1.166667,-0.800000
75%,0.000000,0.266667,-0.055556,-0.222222,0.400000
max,4.166667,5.666667,4.300000,5.600000,5.000000


### Visualizations

In [53]:
# histogram of frequency of input column
inputs = ['age','career_name','goal']

for input in inputs:
    fig = px.histogram(df_uniq_id, x=input, color='gender', barmode='overlay')
    fig.show()
    pyo.plot(fig, filename=input + '_histogram.html')

In [55]:
# pie chart of attendee responses per input
inputs = ['satisfaction', 'length', 'numdat_2']

for input in inputs:
    breakdown_by_input = df_uniq_id.groupby(input)['iid'].count().reset_index()
    fig = px.pie(breakdown_by_input, values='iid', names=input)
    fig.show()
    pyo.plot(fig, filename=input + '_pie.html')

In [57]:
# bar chart of match receive rate by input
inputs = ['age', 'career_name', 'met_o', 'goal']

for input in inputs:
    match_rate_by_input = df_uniq_id.groupby([input, 'gender'])['match_receive_rate'].mean().reset_index()

    fig = px.bar(match_rate_by_input, x=input, y='match_receive_rate', color='gender', color_discrete_sequence=['#FF5733', '#33FF7A'], barmode='overlay')
    fig.show()
    pyo.plot(fig, filename=input + '_match_bar.html')

In [28]:
rating1 = self_rate_day_after
rating2 = self_rate_day_after
rating_diff = {}

for atb in range(0,len(rating1)):
    temp = re.compile("([a-zA-Z]+)([0-9]+)")
    atb_clean = temp.match(rating1[atb]).groups()[0]
    rating_diff[atb_clean + '_diff'] = df[rating2[atb]] - df[rating1[atb]] 

def get_rating_diff(obs):
    for rating in list(rating_diff.keys()):
        print(rating_diff[rating][obs])

#for i in range(0, len(rating_diff)):
#    df[list(rating_diff.keys())[i]] = df.apply(get_rating_diff, axis=1)

'''df.apply(get_rating_diff, axis=1)

df_uniq_id_temp = df.groupby('iid')[list(rating_diff.keys())].mean()

df_uniq_id_temp'''

"df.apply(get_rating_diff, axis=1)\n\ndf_uniq_id_temp = df.groupby('iid')[list(rating_diff.keys())].mean()\n\ndf_uniq_id_temp"

In [29]:
corr_cols = ['shar']
for i in range(0,len(others_ratings)):
    corr_cols.append(others_ratings[i])
target = 'dec_o2'
corr1 = df[corr_cols].corrwith(df[target])

corr_cols2 = ['shar']
for i in range(0,len(scorecard_ratings)):
    corr_cols2.append(scorecard_ratings[i])
target2 = 'match'
corr2 = df[corr_cols2].corrwith(df[target2])

print("Correlations between others' ratings of attendee's attributes and whether or not the other says 'match'\n", corr1)
print("Correlations between attendee's ratings of others' attributes and whether or not they say 'match'\n", corr2)


Correlations between others' ratings of attendee's attributes and whether or not the other says 'match'
 shar       0.270840
attr_o     0.260837
sinc_o     0.164523
intel_o    0.169448
fun_o      0.277700
amb_o      0.139927
dtype: float64
Correlations between attendee's ratings of others' attributes and whether or not they say 'match'
 shar     0.270840
attr     0.260784
sinc     0.164523
intel    0.169448
fun      0.277766
amb      0.139927
dtype: float64


In [30]:
df[df['met'] == df['met_o']]

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,matches_given,dec_o2,matches_received,mutual_match,mutual_count,match_given_rate,match_receive_rate,mutual_rate,career_name,satisfaction
0,1,1.0,0,1,1,1,10,7,NaN,4,...,4,0,4,0,4,0.400000,0.400000,1.0,Other,Somewhat Satisfied
2,1,1.0,0,1,1,1,10,7,NaN,10,...,4,1,4,1,4,0.400000,0.400000,1.0,Other,Somewhat Satisfied
3,1,1.0,0,1,1,1,10,7,NaN,5,...,4,1,4,1,4,0.400000,0.400000,1.0,Other,Somewhat Satisfied
4,1,1.0,0,1,1,1,10,7,NaN,7,...,4,1,4,1,4,0.400000,0.400000,1.0,Other,Somewhat Satisfied
5,1,1.0,0,1,1,1,10,7,NaN,6,...,4,0,4,0,4,0.400000,0.400000,1.0,Other,Somewhat Satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8093,540,10.0,1,20,2,21,22,12,11.0,2,...,1,1,1,1,1,0.045455,0.045455,1.0,Academic/Research,Somewhat Satisfied
8194,544,14.0,1,28,2,21,22,5,5.0,1,...,3,1,3,1,3,0.136364,0.136364,1.0,Doctor/Medicine,Satisfied
8351,551,21.0,1,42,2,21,22,14,21.0,16,...,2,0,2,0,2,0.090909,0.090909,1.0,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Dissatisfied
8352,551,21.0,1,42,2,21,22,13,21.0,15,...,2,0,2,0,2,0.090909,0.090909,1.0,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Dissatisfied


In [31]:
cols_i_want = ['iid','pid','match','dec_o']

for i in range(0,len(scorecard_ratings)):
    cols_i_want.append(scorecard_ratings[i])

for i in range(0,len(scorecard_ratings)):
    cols_i_want.append(others_ratings[i])    

df.query('iid == 412 or pid == 412')[cols_i_want]

,iid,pid,match,dec_o,attr,sinc,intel,fun,amb,attr_o,sinc_o,intel_o,fun_o,amb_o
5759,379,412.0,0,0,5.0,7.0,8.0,7.0,9.0,7.0,8.0,9.0,8.0,7.0
5778,380,412.0,0,1,5.0,6.0,7.0,7.0,6.0,8.0,7.0,8.0,8.0,8.0
5797,381,412.0,0,1,5.0,8.0,7.0,7.0,6.0,7.0,8.0,8.0,7.0,6.0
5816,382,412.0,0,0,5.0,8.0,8.0,8.0,8.0,6.0,8.0,8.0,8.0,7.0
5835,383,412.0,1,1,7.0,7.0,8.0,NaN,NaN,8.0,9.0,9.0,8.0,8.0
5854,384,412.0,1,1,8.0,10.0,10.0,10.0,10.0,8.0,8.0,8.0,7.0,8.0
5873,385,412.0,0,0,6.0,7.0,7.0,7.0,7.0,7.0,8.0,8.0,7.0,7.0
5892,386,412.0,1,1,9.0,9.0,9.0,8.0,NaN,8.0,8.0,8.0,9.0,8.0
5911,387,412.0,1,1,7.0,7.0,8.0,8.0,8.0,9.0,8.0,7.0,9.0,9.0
5930,388,412.0,0,0,5.0,7.0,8.0,7.0,NaN,7.0,6.0,5.0,8.0,8.0


In [48]:
# attendees per career
df_uniq_id['career_name'].value_counts()

Academic/Research                                                       152
Banking/Consulting/Finance/Marketing/Business/CEO/Entrepreneur/Admin    144
Lawyer                                                                   47
Creative Arts/Entertainment                                              46
International/Humanitarian Affairs                                       35
Doctor/Medicine                                                          26
Undecided                                                                25
Other                                                                    17
Psychologist                                                             17
Engineer                                                                 14
Social Work                                                              14
Politics                                                                  5
Real Estate                                                               3
Speech Patho

In [49]:
# attendees by age
df_uniq_id['age'].value_counts()

27.0    68
24.0    56
23.0    56
25.0    55
26.0    55
28.0    47
22.0    44
29.0    40
30.0    36
21.0    22
32.0    13
33.0    12
34.0    11
31.0     7
20.0     5
35.0     4
36.0     4
19.0     2
39.0     1
18.0     1
37.0     1
42.0     1
38.0     1
55.0     1
Name: age, dtype: int64